In [ ]:
from queue import Empty as QueueEmpty
import heapq 

In [1]:
def transactionEngine(stockId, queue, dbQueue, logQueue, users, shutdownEvent):
    def marketTransaction(request):
        global transactions
        side = request.get("side")
        transactionsModified = []
        if side == "buy":
            numberOfStockRequired = request.get("quantity"); amountRecieved = numberOfStockRequired * request.get("pricePerUnit", 0)
            actualPrice = 0.0   # Price while buying stocks
            while numberOfStockRequired > 0 and len(transactions["sell"]) > 0:
                priceInThisTransaction, timeStamp, transaction = heapq.heapop(transactions["sell"])
                stocksInThisTransaction = transaction.get("quantity")   # Stocks that are present with the seller
                addedStocks = min(stocksInThisTransaction, numberOfStockRequired)
                
                stocksInThisTransaction -= addedStocks; numberOfStockRequired -= addedStocks
                thisTransactionValue = addedStocks * priceInThisTransaction
                actualPrice += thisTransactionValue

                _request = {
                    "modify": "partial" if stocksInThisTransaction > 0 else "complete",
                    "sellerId": transaction.get("uId"),
                    "buyerId": request.get("uId"),
                    "buyerTid": request.get("tId"),
                    "sellerTid": request.get("tId"),
                    "amountToSeller": thisTransactionValue, # Amount to be transferred to seller
                    "quantity": addedStocks
                }
                transactionsModified.append(_request)

                if stocksInThisTransaction > 0:
                    # Push the partial transaction back
                    transaction["quantity"] = stocksInThisTransaction   # Modify the number of stocks available
                    heapq.heappush(transactions["sell"], (priceInThisTransaction, timeStamp, transaction))

            if numberOfStockRequired > 0:
                # Add this transaction to the buyers
                request["quantity"] = numberOfStockRequired # Only the quantity changes
                # TO-DO: How to determine the amount the buyer needs to pay (more or less)
                heapq.heappush(transactions["buy"], (actualPrice, request.get("timeStamp"), request))
            
            if transactionsModified:
                # Update the buyer data
                buyerData = users[request.get("uId")]
                
                for transaction in transactionsModified:
                    # Modify the userData for all the users
                    dbQueue.put(transaction)
            
    try:
        transactions = {"buy":[], "sell": []}
        while True:
            if shutdownEvent.is_set():
                break
            try:
                request = queue.get()
            except QueueEmpty as qe:
                pass

            orderType = request.get("orderType")
            if orderType == "limit":
                pass
            elif orderType == "market":
                pass
            elif orderType == "ioc":
                pass
            elif orderType == "pok":
                pass

    except Exception as _e:
        logQueue.put(f"Exception at transaction-engine {stockId}: {str(_e)}")
    
    finally:
        print("Exiting")


2

In [ ]:
import time
st = time.time()
from tinydb import TinyDB

# Open (or create) the database
db = TinyDB('db.json')

# List of transactions (multiple JSON objects like your example)
transactions = [
    {
        "tId": "t12345",
        "uId": "user1",
        "stockId": "btc",
        "side": "buy",
        "orderType": "limit",
        "quantity": 5.0,
        "pricePerUnit": 100.00,
        "status": "RECIEVED",
        "timeStamp": 34234343423
    },
    {
        "tId": "t12346",
        "uId": "user2",
        "stockId": "eth",
        "side": "sell",
        "orderType": "market",
        "quantity": 3.0,
        "pricePerUnit": 200.00,
        "status": "PENDING",
        "timeStamp": 34234343500
    },
    # Add more transactions if you want
]

# Insert multiple transactions at once
db.insert_multiple(transactions)

# Fetch all records
all_transactions = db.all()

# Print all transactions
for tx in all_transactions:
    print(tx)

print(time.time() - st)

Bad pipe message: %s [b' 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Sa']
Bad pipe message: %s [b'ri/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/', b'ng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nAccept-Encoding: gzip, deflate, br, zstd\r\nA']
Bad pipe message: %s [b'ept-Language: en-US,en;q=0.9\r\nPriority: u=0, i\r\nReferer: https://idx.google.com/\r\nSec-Ch-Ua: "Goog', b' Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"\r\nSec-Ch-Ua-Arch: "x86"\r\nSec-Ch-Ua-Bitness: "64"\r', b'ec-Ch-Ua-', b'rm-Factors: "Desktop"\r\nSec-Ch-Ua-Full-Version: "137.0.7151.69"\r\nSec-C', b'Ua-Full-Version-List: "Google Chrome";v="137.0.7151.69", "Chromium";v="137.0.7151.69", "Not/A)Brand";v=', b'4.0.0.0"\r\nSec-Ch-Ua-Mobile: ?0\r\nS']
Bad pipe message: %s [b'-Ch-Ua-Model: ""\r\nSec-Ch-Ua-Platform: "Windows"\r\nSec-Ch-Ua-Platform-Version: "19.0.0"\r\nSec-Ch-Ua-Wow', b': ?0\r\nSec-Fetch-Dest: iframe\r\nSe